In [1]:
import database
import database.chat_history

database.chat_history.delete(user_id="example")

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff0000000000000203'), 'opTime': {'ts': Timestamp(1727254559, 7), 't': 515}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1727254559, 7), 'signature': {'hash': b'g\x8a{g\x92\xe2^\xf12*C\x1c<\r-\xcc\x9c&>\x83', 'keyId': 7366617972116291585}}, 'operationTime': Timestamp(1727254559, 7)}, acknowledged=True)

In [2]:
import tools

tools.set_current_user_id("example2")
tools.get_customer_data(tool_input=[])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


{'_id': ObjectId('66f3c184d82ebda0ef2f1c10'),
 'user_id': 'example2',
 'name': 'หมันหู หมูหัน',
 'age': 97,
 'income_source': None,
 'monthly_income': None,
 'job_status': None,
 'outstanding_loan_amount': None,
 'loan_history': None,
 'missed_payments': None,
 'total_debt_payment_monthly': None,
 'payment_types': None,
 'significant_assets': None}